In [1]:
%pip install h3
%pip install tqdm
%pip install psycopg[binary]

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import h3
import tqdm
import json
import numpy as np
import pandas as pd
import asyncio
import psycopg

### Calculate Transit Accessibility Scores
#### Cell types:
* `-1`: geo cell, no station here
* ` 0`: bus stop
* ` 1`: tram / ferry stop
* ` 2`: metro / s-tog station
* ` 3`: train station


In [3]:
H3_RES = 11
HOURS = 24 * 7
TRANSIT_TYPE = 3
MAX_DISTANCE = 10

In [4]:
DB_CONN = "postgresql://postgres:byS*<7AxwYC#U24s@srv-captain--postgres-db-db/postgres"

In [ ]:
# for development only, select a specific h3_4 cell
SELECTED_H3_4 = '841f059ffffffff'
SELECTED_H3_4_QUERY = f"and h3_4 = '{SELECTED_H3_4}'"

#### Load entrypoint (station) data

In [5]:
# load all stops for a specific type from the database
async with await psycopg.AsyncConnection.connect(DB_CONN) as conn:
    async with conn.cursor() as cursor:
        await cursor.execute(f"""
        select * from transit
            where type = 3
            {SELECTED_H3_4_QUERY}
        """)

        stations = await cursor.fetchall()

In [6]:
# add all stations to a queue
queue = asyncio.Queue()
for station in stations:
    # (h3 cell, scores array, distance)
    await queue.put((station[0], station[3], 0))

#### Functions

In [7]:
# define formula here
async def calc_score(origin_scores, distance, type):
    result_scores = []
    for score in origin_scores:
        if score > 1 and distance < MAX_DISTANCE:
            result_scores.append(score - 1)
        else:
            result_scores.append(0)
    return result_scores

In [8]:
async def update_scores(origin_h3, scores, distance):
    print(f"calculating {origin_h3}, dist={distance}")
    # get h3 k-ring for origon_h3
    neighbors = h3.k_ring(origin_h3, k= 1)
    neighbors.discard(origin_h3)

    # get neighbors from DB
    async with await psycopg.AsyncConnection.connect(DB_CONN) as conn:
        async with conn.cursor() as cursor:
            await cursor.execute("""
                select h3 from transit
                    where h3 = any(%s)
                    and cardinality(scores) = 0
            """, [list(neighbors)])

            db_neighbors = await cursor.fetchall()

            # update scores for neighbors according to calc_score function
            for neighbor in db_neighbors:
                neighbor = neighbor[0]

                score = await calc_score(scores, distance + 1, TRANSIT_TYPE)
                await cursor.execute(f"""
                    update transit
                    set scores = array_cat(scores, array {score})
                    where h3 = '{neighbor}'
                """)

                if sum(score) > 0 and distance < MAX_DISTANCE:
                    await queue.put((neighbor, score, distance + 1))
        await conn.commit()

#### Execution Loop

In [9]:
while not queue.empty():
    current = await queue.get()
    await asyncio.gather(
        update_scores(current[0], current[1], current[2])
    )

calculating 8b1f05832686fff, dist=0
calculating 8b1f05901413fff, dist=0
calculating 8b1f05831152fff, dist=0
calculating 8b1f05903ad9fff, dist=0
calculating 8b1f05905b36fff, dist=0
calculating 8b1f0592e781fff, dist=0
calculating 8b1f0592eb8bfff, dist=0
calculating 8b1f05900262fff, dist=0
calculating 8b1f059054c0fff, dist=0
calculating 8b1f05905106fff, dist=0
calculating 8b1f0581882efff, dist=0
calculating 8b1f05811582fff, dist=0
calculating 8b1f05811748fff, dist=0
calculating 8b1f059b2a0afff, dist=0
calculating 8b1f05833b62fff, dist=0
calculating 8b1f05832103fff, dist=0
calculating 8b1f05981932fff, dist=0
calculating 8b1f0592d45dfff, dist=0
calculating 8b1f0590d420fff, dist=0
calculating 8b1f05824cb1fff, dist=0
calculating 8b1f0582278cfff, dist=0
calculating 8b1f05832680fff, dist=1
calculating 8b1f05832682fff, dist=1
calculating 8b1f05832684fff, dist=1
calculating 8b1f05832695fff, dist=1
calculating 8b1f058326b1fff, dist=1
calculating 8b1f058326b3fff, dist=1
calculating 8b1f05901410fff,

In [ ]:
# load the result
async with await psycopg.AsyncConnection.connect(DB_CONN) as conn:
    async with conn.cursor() as cursor:
        await cursor.execute(f"""
        select * from transit
            where type = any(-1, {TRANSIT_TYPE})
            {SELECTED_H3_4_QUERY}
        """)

        stations = await cursor.fetchall()